<a href="https://colab.research.google.com/github/jeffersondemota/projeto_/blob/main/Calculo_Distancia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================
# DISTÂNCIA POR ESTRADA (OSRM público) -> Excel de saída
# Google Colab: faz upload do .xlsx e baixa o resultado
# ============================================================

!pip -q install openpyxl requests

import pandas as pd
import requests
import time
import math
from google.colab import files



In [2]:
# ---------- CONFIG ----------
lat_origem = -23.5926917
lon_origem = -46.8774224

OSRM_BASE_URL = "https://router.project-osrm.org"  # servidor público gratuito
PROFILE = "driving"  # pode trocar para "driving", "car" não existe no OSRM público
SLEEP_BETWEEN_REQUESTS = 0.25  # limite simples (segundos)
MAX_RETRIES = 3
TIMEOUT_SECONDS = 15



In [3]:
# ---------- FUNÇÕES ----------
def is_valid_coord(lat, lon):
    try:
        if pd.isna(lat) or pd.isna(lon):
            return False
        lat = float(lat)
        lon = float(lon)
        return (-90.0 <= lat <= 90.0) and (-180.0 <= lon <= 180.0) and not (math.isnan(lat) or math.isnan(lon))
    except Exception:
        return False

def osrm_distance_km(lat_o, lon_o, lat_d, lon_d, session: requests.Session):
    """
    Retorna distância em km (float) via OSRM route.
    """
    # OSRM usa formato: lon,lat
    url = f"{OSRM_BASE_URL}/route/v1/{PROFILE}/{lon_o},{lat_o};{lon_d},{lat_d}"
    params = {
        "overview": "false",
        "alternatives": "false",
        "steps": "false"
    }
    r = session.get(url, params=params, timeout=TIMEOUT_SECONDS)
    r.raise_for_status()
    data = r.json()

    if data.get("code") != "Ok":
        raise RuntimeError(f"OSRM retornou code={data.get('code')}, message={data.get('message')}")
    routes = data.get("routes", [])
    if not routes:
        raise RuntimeError("OSRM não retornou rotas.")
    dist_m = routes[0].get("distance", None)  # metros
    if dist_m is None:
        raise RuntimeError("Distância não encontrada na resposta do OSRM.")
    return dist_m / 1000.0



In [4]:
# ---------- UPLOAD ----------
print("📤 Faça upload do arquivo .xlsx base:")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("Nenhum arquivo foi enviado.")

input_filename = list(uploaded.keys())[0]
print(f"Arquivo recebido: {input_filename}")



📤 Faça upload do arquivo .xlsx base:


Saving _RelatorioCadastroCliente_.xls to _RelatorioCadastroCliente_.xls
Arquivo recebido: _RelatorioCadastroCliente_.xls


In [5]:
# ---------- LEITURA ----------
df = pd.read_excel(input_filename)

# Validar colunas obrigatórias
required = ["Latitude", "Longitude"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Faltam colunas obrigatórias na planilha: {missing}")

# Criar Distancia_Km se não existir
if "Distancia_Km" not in df.columns:
    df["Distancia_Km"] = pd.NA

# ---------- PROCESSAMENTO ----------
total = len(df)
ok = 0
fail = 0
skipped_invalid = 0

session = requests.Session()

for idx, row in df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]

    # Se inválido, deixa vazio e continua
    if not is_valid_coord(lat, lon):
        df.at[idx, "Distancia_Km"] = pd.NA
        skipped_invalid += 1
        continue

    lat = float(lat)
    lon = float(lon)

    # retries em falhas temporárias
    success = False
    last_err = None

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            dist_km = osrm_distance_km(lat_origem, lon_origem, lat, lon, session)
            df.at[idx, "Distancia_Km"] = round(dist_km, 3)
            ok += 1
            success = True
            break
        except (requests.exceptions.Timeout,
                requests.exceptions.ConnectionError,
                requests.exceptions.HTTPError,
                RuntimeError) as e:
            last_err = e
            # backoff simples
            time.sleep(SLEEP_BETWEEN_REQUESTS + attempt * 0.5)

    if not success:
        df.at[idx, "Distancia_Km"] = pd.NA
        fail += 1

    time.sleep(SLEEP_BETWEEN_REQUESTS)

# ---------- SAÍDA ----------
if input_filename.lower().endswith(".xlsx"):
    output_filename = input_filename[:-5] + "_com_distancia.xlsx"
else:
    output_filename = input_filename + "_com_distancia.xlsx"

df.to_excel(output_filename, index=False)

print("\n✅ Concluído!")
print(f"Total de linhas: {total}")
print(f"Calculadas com sucesso: {ok}")
print(f"Falharam (OSRM/HTTP): {fail}")
print(f"Puladas por Latitude/Longitude inválidas/vazias: {skipped_invalid}")
print(f"Arquivo gerado: {output_filename}")

# ---------- DOWNLOAD ----------
files.download(output_filename)



✅ Concluído!
Total de linhas: 48
Calculadas com sucesso: 48
Falharam (OSRM/HTTP): 0
Puladas por Latitude/Longitude inválidas/vazias: 0
Arquivo gerado: _RelatorioCadastroCliente_.xls_com_distancia.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>